p＝

In [25]:
using BasisMatrices
using QuantEcon

In [26]:
# set environment
n = 8
smin = 0.7
smax = 1.3
maxit = 1000
tol = 0.0001

# make collocation matrix
sgrid0 = linspace(smin, smax, n)
basis = Basis(SplineParams(sgrid0, 0, 3), SplineParams(sgrid0, 0, 3))
S, (coordx, coordy) = nodes(basis)
Φ = BasisMatrix(basis, Expanded(), S, 0)

BasisMatrix{BasisMatrices.Expanded} of order [0 0]

In [27]:
mutable struct CPG
    alpha::Array{Float64,1}
    beta::Array{Float64,1}
    gamma::Array{Float64,1}
    psi::Float64
    delta::Float64
end


function cost( model::CPG, s::Array{Float64, 2})
    box=zeros(size(S)[1], 2)
    box[:,1]= model.beta[1]+( model.beta[2] ./ s[:,1])
    box[:,2]= model.beta[1]+( model.beta[2] ./ s[:,2])
    return box
end

cost (generic function with 1 method)

In [28]:
function prof( model::CPG, cost::Array{Float64, 2})
    return (model.alpha[1]-2*cost[:,1]+cost[:,2].^2)/(9*model.alpha[2])
end

prof (generic function with 1 method)

In [29]:
function reward( model::CPG, prof::Array{Float64, 1},x::Array{Float64, 2}, p::Int64)     
    return prof-(model.gamma[1]*x[:,p]+0.5*model.gamma[2]*x[:,p].^2)
end

reward (generic function with 1 method)

In [30]:
function capital( model::CPG, x::Array{Float64, 2}, s::Array{Float64, 2})     
    return (1-psi)*s + x
end

capital (generic function with 1 method)

In [31]:
# derivatives of  payoff, transition functions
# reward function

function rewardx( model::CPG, x::Array{Float64, 2}, s::Array{Float64, 2}) 
    n = size(s)[1]
    fx = zeros(n,2);
    fx[:,1] = -(model.gamma[1]+model.gamma[2]*x[:,1]);
    return fx
end

rewardx (generic function with 1 method)

In [32]:
function rewardxx( model::CPG, x::Array{Float64, 2}, s::Array{Float64, 2}) 
    n = size(s)[1]
    fxx= zeros(n,2,2)-gamma[2];
    fxx[:,1,1] = zeros(n,1)-gamma[2];
    fxx[:,2,2] = zeros(n,1)-gamma[2];
    return fxx
    end

rewardxx (generic function with 1 method)

In [33]:
# transition function
function capitalx( model::CPG, x::Array{Float64, 2}, s::Array{Float64, 2}) 
    n = size(s)[1]
    gx = zeros(n,2,2);
    gx[:,1,1] = ones(n,1);
    gx[:,2,2] = ones(n,1);
    return gx
end

capitalx (generic function with 1 method)

In [34]:
function capitalxx( model::CPG, x::Array{Float64, 2}, s::Array{Float64, 2}) 
    n = size(s)[1]
    gxx = zeros(n,2,2,2)
    return gxx
end

capitalxx (generic function with 1 method)

In [35]:
# collocation function
function vmax(model::CPG, colnodes::Array{Float64, 2}, b, action::Array{Float64, 2}, coef::Array{Float64, 2})
    xnew = action
    v = zeros((size(colnodes)[1], 2))
    c = cost(model, colnodes)
    pr = prof(model, c)
    for p in 1:2
        xl, xu = 0.0, Inf*ones(n,2)
        order1 = [0 0]
        order1[1, p] = 1
        order2 = [0 0]
        order2[1, p] = 2
        for it in 1:1
            util, util_der1, util_der2 = reward(model, pr, action, p), rewardx(model, action, colnodes), rewardxx(model, action, colnodes) 
            Ev, Evx, Evxx = 0.0, 0.0, 0.0
            transition, transition_der1, transition_der2 = capital(model, action, colnodes), capitalx(model, action, colnodes), capitalxx(model, action, colnodes)
            vn = funeval(coef[:, p], b, transition)
            vnder1 =  funeval(coef[:, p], b, transition, order1)
            vnder2 = funeval(coef[:, p], b, transition, order2)
            Ev =   vn
            Evx =  vnder1.* transition_der1
            Evxx =  (vnder1.*transition_der2 + vnder2 .* (transition_der1.^2))
            v[:, p] = util + Ev
            delx = -(util_der1 + model.delta * Evx) ./ (util_der2 + model.delta*Evxx)
            delx = min.(max.(delx, xl-action[:, p]), xu-action[:, p])
            action[:, p] = action[:, p] + delx
            if norm(delx) < tol
                break
            end
        end
        xnew[:, p] = action[:, p]
    end
    return v, xnew
end

vmax (generic function with 1 method)

In [36]:
psi = 0.1;
delta = 0.9;
alpha = [8.0 , 4.0];
beta  = [1.8 ,0.2];
gamma = [0.4, 3.0];
model=CPG(alpha, beta, gamma, psi, delta)

CPG([8.0, 4.0], [1.8, 0.2], [0.4, 3.0], 0.1, 0.9)

In [37]:
# set initials
# num_nodesが3ならsrand(1234)
srand(12347)#色々試す
initial = rand((size(Φ.vals[1])[1]), 2)
x = zeros((size(Φ.vals[1])[1]), 2)+1
v = zeros((size(Φ.vals[1])[1]), 2)
model = CPG([8.0,4.0], [1.8, 0.2],[0.4,3.0], 0.1, 0.9)

CPG([8.0, 4.0], [1.8, 0.2], [0.4, 3.0], 0.1, 0.9)

In [38]:
vmax(model,S,basis,x,initial)

LoadError: [91mDimensionMismatch("dimensions must match")[39m

In [46]:
# set initials
srand(77)
initial = rand((size(Φ.vals[1])[1]), 2)
x = zeros((size(Φ.vals[1])[1]), 2)+1
v = zeros((size(Φ.vals[1])[1]), 2)
model = CPG([8.0,4.0], [1.8, 0.2],[0.4,3.0], 0.1, 0.9)

# iteration for coefficients
c = initial
c_error = c
count = 0
for it in 1:10000
    cold = c
    vnew, x = vmax(model, S, basis, x, c, e, w)
    c = Φ.vals[1] \ vnew
    v = vnew
    c_error = cold - c
    count += 1
    if maximum(abs, cold - c) < tol
        break
    end
end

println(count)
println(c_error)

LoadError: [91mUndefVarError: w not defined[39m